In [102]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

iris = fetch_ucirepo(id=53)

X = iris.data.features
y = iris.data.targets
df = iris.data.original

label_dict = {k: 0 for k in y['class']}


In [103]:
import numpy as math

def computeDistance(a, b):
    return math.sqrt(math.sum([(ai-bi) ** 2 for ai, bi in zip(a, b)]))


In [104]:
def getMajorityLabel(neighboursList):
    majority = label_dict.copy()
    furthest = neighboursList[-1][0] + 1
    for dist, label in neighboursList:
        majority[label] += furthest - dist
    return max(majority, key=majority.get)


In [105]:
def computeStrata(Y):
    result = label_dict.copy()

    for k in result:
        result[k] = len([yi for yi in Y['class'] if yi==k])/len(Y)
    
    return result


In [106]:
def simpleSample(data):
    testing = data.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.2))
    learning = data.drop(testing.index)
    return learning, testing


In [115]:
def fold10Sample(data):
    folds = []
    for i in range(0,10):
        folds.append(data.groupby('class', group_keys=False).apply(lambda x: x.sample(5)))
        data.drop(folds[i].index)
    return folds


In [107]:
def kNNClassify(X, Y, xt, k):
    distancePairList = []
    for xi, yi in zip(X.values, Y.values):
        di = computeDistance(xi, xt)
        distancePairList.append((di, yi))
    distancePairList.sort()

    kNeighboursList = distancePairList[:k]
    yt = getMajorityLabel(kNeighboursList)

    return yt


In [ ]:
import numpy as np

learning, testing = simpleSample(df)
k = input("Value?")
k = int(k)
success = 0
for t in testing.values:
    #print(learning.drop('class', axis=1), learning['class'])
    ys = kNNClassify(learning.drop('class', axis=1), learning['class'], t[:-1], k )
    if ys == t[-1]:
        success += 1

print("Train Set Accuracy:",success/30*100)

folds = fold10Sample(df)
successes = [0] * 10
for i in range(0,10):
    learning = df.drop(folds[i].index)
    for t in folds[i].values:
        ys = kNNClassify(learning.drop('class', axis=1), learning['class'], t[:-1], k )
        if ys == t[-1]:
            successes[i] += 1
    successes[i] = successes[i]/15*100
    print("Accuracy Fold", i+1, ":",successes[i])

print("\nAverage Accuracy:", np.average(successes))
print("Standard Deviation:", np.std(successes))
#print(computeDistance(sample.values[0][:-1], sample.values[1][:-1]))


C:\Users\bugyb\AppData\Local\Temp\ipykernel_2296\2761867053.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  testing = data.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.2))


Train Set Accuracy: 193.33333333333334
Accuracy Fold 1 : 100.0
Accuracy Fold 2 : 100.0
Accuracy Fold 3 : 100.0
Accuracy Fold 4 : 86.66666666666667
Accuracy Fold 5 : 100.0
Accuracy Fold 6 : 100.0


C:\Users\bugyb\AppData\Local\Temp\ipykernel_2296\1620848994.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  folds.append(data.groupby('class', group_keys=False).apply(lambda x: x.sample(5)))
C:\Users\bugyb\AppData\Local\Temp\ipykernel_2296\1620848994.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  folds.append(data.groupby('class', group_keys=False).apply(lambda x: x.sample(5)))
C:\Users\bugyb\AppData\Local\T

Accuracy Fold 7 : 100.0
Accuracy Fold 8 : 100.0
Accuracy Fold 9 : 93.33333333333333
Accuracy Fold 10 : 100.0

Average Accuracy: 98.00000000000001
Standard Deviation: 4.268749491621898
